# Load Data

In [2]:
import numpy as np
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import cv2
import random
import os
import numpy as np

In [3]:
anno_path_box = "./dataset/Annotations/daySequence1/frameAnnotationsBOX.csv"
anno_path_bulb = "./dataset/Annotations/daySequence1/frameAnnotationsBULB.csv"
frames_path = "./dataset/daySequence1/frames/"

images = []
labels = []
with open(anno_path_box) as fp:  
    line = fp.readline()
    line = fp.readline() # Skip header line with descriptions

    while line:
        anno_file_path = (line.strip()).split(";")
        anno_file_id = anno_file_path[0].split("/")[1]
        
        file_name = os.path.join(os.path.join(frames_path, anno_file_id))
        
        anno_left = int(anno_file_path[2])
        anno_top = int(anno_file_path[3])
        anno_right = int(anno_file_path[4])
        anno_bot = int(anno_file_path[5])
        
        file = cv2.imread(file_name)
        file = cv2.cvtColor(file, cv2.COLOR_BGR2RGB);
        crop = file[anno_top:anno_bot, anno_left:anno_right]
        resized = cv2.resize(crop, (16,24))
        
        images.append(resized)
        labels.append(
            {'go': 0,
             'goLeft': 0,
             'goForward': 0,
             'warning': 1,
             'warningLeft': 1,
             'stop': 2,
             'stopLeft': 2
            }[anno_file_path[1]]
        )
        
        line = fp.readline()

In [13]:
import numpy as np
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(np.array(images), np.array(labels), test_size=0.33, random_state=42)

In [14]:
n_train = len(X_train)
n_test = len(X_test)
image_shape = X_train[0].shape
n_classes = len(np.unique(y_train))

print("Number of training examples =", n_train)
print("Number of testing examples =", n_test)
print("Image data shape =", image_shape)
print("Number of classes =", n_classes)

Number of training examples = 5293
Number of testing examples = 2607
Image data shape = (24, 16, 3)
Number of classes = 3


# Train NN

In [15]:
from keras.models import Sequential
from keras.layers import Conv2D, Flatten, Dense, MaxPooling2D, Dropout
from keras.utils.np_utils import to_categorical
from keras import losses, optimizers, regularizers

In [26]:
num_classes = 3
model = Sequential()
model.add(Conv2D(16, (3, 3), input_shape=(24, 16, 3), padding='same', activation='relu', kernel_initializer='random_uniform', kernel_regularizer=regularizers.l2(0.01)))
model.add(MaxPooling2D(2,2))
#Dropout(0.8)
#model.add(Conv2D(32, (3, 3), padding='same', activation='relu', kernel_initializer='random_uniform', kernel_regularizer=regularizers.l2(0.01)))
#model.add(MaxPooling2D(2,2))
Dropout(0.2)
model.add(Flatten())

In [29]:
loss = losses.categorical_crossentropy
optimizer = optimizers.Adam()

In [30]:
model.compile(loss=loss, optimizer=optimizer, metrics=['accuracy'])

In [32]:
print(y_train.shape)

(5293,)


In [23]:
model.fit(X_train, y_train, batch_size=64, epochs=10, verbose=True, validation_split=0.1, shuffle=True)

ValueError: Error when checking target: expected flatten_3 to have shape (1536,) but got array with shape (1,)